In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import tensorflow as tf
import tensorflow.keras as kr
from tqdm.notebook import tqdm
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import json

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
song_meta = pd.read_json('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/멜론데이터/song_meta.json', typ = 'frame')
train = pd.read_json('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/멜론데이터/train.json', typ = 'frame')
val = pd.read_json('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/멜론데이터/val.json', typ = 'frame')

In [ ]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [ ]:
train_data = train[['id', 'songs','like_cnt']]
train_data_unnest = np.dstack((np.repeat(train_data.id.values, list(map(len, train_data.songs))),
                               np.concatenate(train_data.songs.values),
                               np.repeat(train_data.like_cnt.values, list(map(len, train_data.songs)))))
train_data = pd.DataFrame(data = train_data_unnest[0], columns = train_data.columns)
train_data['id'] = train_data['id'].astype(int)
train_data['songs'] = train_data['songs'].astype(int)
train_data['like_cnt'] = train_data['like_cnt'].astype(int)
train_data.columns = ['listid','songid', 'likecnt']

del train_data_unnest
train_data

,listid,songid,likecnt
0,61281,525514,71
1,61281,129701,71
2,61281,383374,71
3,61281,562083,71
4,61281,297861,71
...,...,...,...
5285866,100389,111365,17
5285867,100389,51373,17
5285868,100389,640239,17
5285869,100389,13759,17


In [ ]:
val_data = val[['id', 'songs', 'like_cnt']]
val_data_unnest = np.dstack((np.repeat(val_data.id.values, list(map(len, val_data.songs))),
                             np.concatenate(val_data.songs.values),
                             np.repeat(val_data.like_cnt.values, list(map(len, val_data.songs)))))
val_data = pd.DataFrame(data = val_data_unnest[0], columns = val_data.columns)
val_data['id'] = val_data['id'].astype(int)
val_data['songs'] = val_data['songs'].astype(int)
val_data['like_cnt'] = val_data['like_cnt'].astype(int)
val_data.columns = ['listid','songid', 'likecnt']

del val_data_unnest
val_data

,listid,songid,likecnt
0,118598,373313,1675
1,118598,151080,1675
2,118598,275346,1675
3,118598,696876,1675
4,118598,165237,1675
...,...,...,...
421194,65189,193899,19
421195,65189,398886,19
421196,65189,234875,19
421197,65189,243850,19


In [ ]:
playlist_map = pd.concat([train_data, val_data], ignore_index=True)
playlist_map

,listid,songid,likecnt
0,61281,525514,71
1,61281,129701,71
2,61281,383374,71
3,61281,562083,71
4,61281,297861,71
...,...,...,...
5707065,65189,193899,19
5707066,65189,398886,19
5707067,65189,234875,19
5707068,65189,243850,19


In [ ]:
like_count = playlist_map.groupby('songid').sum()['likecnt']
(like_count.sort_values(ascending = False) > 60000).sum()

1859

In [ ]:
like_count_60000 = pd.DataFrame(like_count)
like_count_60000.columns = ['likecount']
like_count_60000 = like_count_60000[like_count_60000['likecount'] >= 60000].sort_values(ascending = False, by = 'likecount')
like_count_60000.reset_index(inplace = True)
like_count_60000['songnewid'] = range(len(like_count_60000))
like_count_60000

,songid,likecount,songnewid
0,366786,352819,0
1,133143,277721,1
2,625875,277206,2
3,610933,277176,3
4,580074,272594,4
...,...,...,...
1854,43689,60142,1854
1855,409828,60123,1855
1856,418694,60103,1856
1857,349306,60052,1857


In [ ]:
song_meta_60000 = song_meta[song_meta['id'].isin(like_count_60000['songid'])]
song_meta_60000.rename(columns={'id': 'songid'}, inplace=True)
song_meta_60000 = pd.merge(song_meta_60000, like_count_60000, on = 'songid', how = 'left')
song_meta_60000

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,songid,likecount,songnewid
0,"[GN1304, GN1301]",20110810,Simple Steps,2058451,[614091],Rollercoaster,[GN1300],[Sam Ock],71,66252,1502
1,"[GN0105, GN0101]",20130404,The 3rd Album Part.2 `Love Blossom (러브블러썸)`,2180379,[175139],Love Blossom (러브블러썸),[GN0100],[케이윌],250,130794,246
2,"[GN0401, GN0403, GN0402]",20151012,No Make Up,2644221,[537920],No Make Up,[GN0400],[Zion.T],311,81350,951
3,"[GN1301, GN1302]",20070927,L-O-V-E,358953,[53223],L-O-V-E,[GN1300],[Joss Stone],470,79724,994
4,"[GN2207, GN1501, GN1506, GN1509]",20010718,센과 치히로의 행방불명 OST,2291981,[2796],あの夏へ / Ano Natsue (어느 여름날),"[GN1500, GN2200]",[Hisaishi Joe],1133,69128,1368
...,...,...,...,...,...,...,...,...,...,...,...
1854,"[GN1701, GN1703, GN1702]",20020301,Come Away With Me,22183,[28509],Don`t Know Why,[GN1700],[Norah Jones],705515,200293,33
1855,"[GN0105, GN0101]",20110311,잠 못 드는 밤에,1199953,[4699],잠 못 드는 밤에,[GN0100],[문명진],707564,85700,852
1856,[GN0901],20150904,WILD,2638374,[780919],FOOLS,[GN0900],[Troye Sivan],707573,106095,501
1857,"[GN0104, GN0101]",20080819,김범수 6집,393851,[6502],슬픔활용법,[GN0100],[김범수],707621,86021,845


In [ ]:
playlist_map_60000 = playlist_map[playlist_map['songid'].isin(like_count_60000['songid'])]
playlist_map_60000

,listid,songid,likecnt
21,10532,497066,1
28,10532,532114,1
29,10532,586541,1
43,10532,6546,1
44,10532,152422,1
...,...,...,...
5707056,65189,362966,19
5707057,65189,144663,19
5707062,65189,581789,19
5707064,65189,701557,19


In [ ]:
playlist_map_60000_newid = pd.merge(playlist_map_60000, like_count_60000, on = 'songid', how = 'left')
del playlist_map_60000_newid['songid']
del playlist_map_60000_newid['likecnt']
playlist_map_60000_newid

,listid,likecount,songnewid
0,10532,207995,26
1,10532,152699,125
2,10532,72717,1233
3,10532,130333,251
4,10532,103515,537
...,...,...,...
903098,65189,64524,1603
903099,65189,173208,69
903100,65189,68308,1399
903101,65189,130391,250


In [ ]:
num_song = playlist_map_60000_newid.songnewid.nunique()
print(num_song)

1859


In [ ]:
playlist_map_60000_newid_train = playlist_map_60000_newid[
    playlist_map_60000_newid['listid'].isin(train['id'])]
playlist_map_60000_newid_val = playlist_map_60000_newid[playlist_map_60000_newid['listid'].isin(val['id'])]

In [ ]:
num_user_train = playlist_map_60000_newid_train.listid.nunique()
num_user_val = playlist_map_60000_newid_val.listid.nunique()
print(num_user_train)
print(num_user_val)

84596
12053


In [ ]:
playlist_map_60000_newid_trainlog = playlist_map_60000_newid_train
playlist_map_60000_newid_trainlog['likecount'] = np.log(playlist_map_60000_newid_train['likecount'])
playlist_map_60000_newid_vallog = playlist_map_60000_newid_val
playlist_map_60000_newid_vallog['likecount'] = np.log(playlist_map_60000_newid_val['likecount'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
playlist_map_60000_newid_trainlog.to_csv('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/playlist_map_60000_newid_trainlog.csv')
playlist_map_60000_newid_vallog.to_csv('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/playlist_map_60000_newid_vallog.csv')

In [ ]:
playlist_map_60000_newid_trainlog = pd.read_csv('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/playlist_map_60000_newid_trainlog.csv')
playlist_map_60000_newid_vallog = pd.read_csv('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/playlist_map_60000_newid_vallog.csv')

In [ ]:
song_meta_60000.to_csv('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/song_meta_60000.csv')

In [ ]:
user_dict_train = {}
song_dict_train = {}
n_mapped_user_train = 0
n_mapped_song_train = 0
R_train = np.zeros((num_user_train, num_song))
for index, row in tqdm(playlist_map_60000_newid_trainlog.iterrows()):
  if row.listid in user_dict_train:
    user_cur = user_dict_train[row.listid]
  else:
    user_cur = n_mapped_user_train
    n_mapped_user_train += 1
    user_dict_train[row.listid] = user_cur
  if row.songnewid in song_dict_train:
    song_cur = song_dict_train[row.songnewid]
  else:
    song_cur = n_mapped_song_train
    n_mapped_song_train += 1
    song_dict_train[row.songnewid] = song_cur
  R_train[user_cur, song_cur] = row.likecount

0it [00:00, ?it/s]

In [ ]:
user_dict_val = {}
song_dict_val = {}
n_mapped_user_val = 0
n_mapped_song_val = 0
R_val = np.zeros((num_user_val, num_song))
for index, row in tqdm(playlist_map_60000_newid_vallog.iterrows()):
  if row.listid in user_dict_val:
    user_cur = user_dict_val[row.listid]
  else:
    user_cur = n_mapped_user_val
    n_mapped_user_val += 1
    user_dict_val[row.listid] = user_cur
  if row.songnewid in song_dict_val:
    song_cur = song_dict_val[row.songnewid]
  else:
    song_cur = n_mapped_song_val
    n_mapped_song_val += 1
    song_dict_val[row.songnewid] = song_cur
  R_val[user_cur, song_cur] = row.likecount

0it [00:00, ?it/s]

In [ ]:
R_train_tr, R_train_te = train_test_split(R_train, test_size = 0.1)

In [ ]:
input_ = kr.layers.Input(shape=R_train.shape[1:])
hidden1 = kr.layers.Dense(600, activation='relu', kernel_regularizer=kr.regularizers.l2(0.001))(input_)
hidden2 = kr.layers.Dense(128, activation='relu', kernel_regularizer=kr.regularizers.l2(0.001))(hidden1)
hidden3 = kr.layers.Dense(600, activation='relu', kernel_regularizer=kr.regularizers.l2(0.001))(hidden2)
output_ = kr.layers.Dense(R_train.shape[1], kernel_regularizer=kr.regularizers.l2(0.01))(hidden3)
model = kr.Model(inputs=[input_], outputs=[output_])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(R_train_tr, R_train_tr, validation_data=(R_train_te, R_train_te), epochs=10)

Epoch 1/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1188.7885 - accuracy: 0.0155 - val_loss: 1289.1746 - val_accuracy: 0.0178
Epoch 2/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1158.6384 - accuracy: 0.0172 - val_loss: 1170.8872 - val_accuracy: 0.0178
Epoch 3/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1151.7272 - accuracy: 0.0172 - val_loss: 1018.5156 - val_accuracy: 0.0178
Epoch 4/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1130.4246 - accuracy: 0.0172 - val_loss: 1145.6025 - val_accuracy: 0.0178
Epoch 5/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1168.0638 - accuracy: 0.0178 - val_loss: 1278.3892 - val_accuracy: 0.0184
Epoch 6/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1177.8348 - accuracy: 0.0183 - val_loss: 1231.3533 - val_accuracy: 0.0189
Epoch 7/10
2380/2380 [==============================] - 27s 11ms/step - loss: 1146.0544 

In [ ]:
model.save('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/simple_40000b.h5')

In [ ]:
model = load_model('/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/오토인코더_w2v_앙상블/simple_40000b.h5')

In [ ]:
a = pd.DataFrame(data=model.predict(R_val))
a

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858
0,24.253807,-51.254528,-46.996181,22.569067,-71.503616,13.388608,12.332981,6.001461,13.366298,106.301331,31.064516,-98.443832,22.532354,57.792255,6.171138,9.599284,13.308151,-16.490648,11.675436,-27.039684,-31.215252,14.863453,11.425688,-17.813820,10.416659,-17.626663,11.600269,10.094570,11.633101,9.992886,72.556824,-62.836308,-70.312004,29.056932,-74.977005,60.691566,34.763988,20.759298,30.082634,16.844210,...,14.567137,88.178322,11.672685,-31.285986,32.076256,12.897902,57.011841,-59.644859,19.809523,-59.447575,12.047776,71.056282,-32.886795,10.073548,10.150181,-42.986034,-10.409890,-16.020340,3.109059,10.676986,6.262126,-45.910763,7.908444,12.076251,-21.100864,-35.184357,-27.313522,-53.212902,9.438646,42.398090,-30.230003,11.597405,-31.957111,6.898540,-46.485683,25.765547,12.713105,-39.970078,6.378341,-47.855728
1,-1470.929565,-1355.127563,-920.566895,1043.830811,-1586.635742,615.759155,542.872253,606.382629,633.666504,-907.982788,1109.625977,502.574860,-1640.494019,547.377014,455.903229,460.610596,671.367065,-1077.143066,465.658203,423.840729,-1224.640991,806.708252,537.316895,-1166.401123,562.356445,-1081.201660,556.476074,458.086487,585.741516,513.637329,-1105.975342,-1320.177734,-733.267273,638.273254,-1248.418701,-1627.323120,1229.539185,1211.728271,1166.943237,789.202148,...,565.284180,-395.697205,367.758148,305.977081,532.977600,513.626282,-511.950928,-605.787354,413.908905,-439.391907,517.157349,-690.811340,305.548248,375.504303,440.933838,-538.505127,-476.300964,-294.868561,361.701752,386.444885,415.353729,-345.264374,305.618561,-57.798676,-376.746918,-351.272430,-416.032013,-165.370071,279.384796,-234.541306,-182.555313,326.258575,348.598480,308.789825,126.894943,-174.333984,250.836243,145.915894,233.100632,162.703857
2,38.976582,-77.060745,-66.863953,39.218273,-104.646858,24.447001,23.030571,13.239692,24.569788,163.471664,51.005142,-139.776993,33.790161,89.366989,13.850610,18.779446,24.210264,-23.195904,20.307323,-35.579529,-44.954159,26.796583,21.443331,-26.152349,19.982233,-24.870432,21.692665,19.328588,21.963381,19.211355,111.385628,-92.920868,-101.624939,47.313442,-110.589973,91.345940,57.137215,36.306488,50.075703,30.292643,...,26.070877,134.185013,21.766052,-41.784912,51.717644,23.511215,89.554932,-84.633240,33.372822,-86.112328,22.775463,108.050591,-43.664967,19.303455,19.264071,-61.396000,-13.067863,-20.675570,8.816784,19.884951,13.307994,-65.717804,15.889306,22.052849,-28.253523,-52.007095,-40.865597,-76.187973,19.670727,68.342125,-42.743073,21.092951,-43.820976,14.198607,-65.744553,43.105110,22.443024,-54.794926,13.540292,-68.183197
3,-2371.488281,-2173.900146,-1482.632202,1668.704712,-2550.095947,980.166443,864.138245,966.156799,1011.174316,-1472.317993,1770.925781,804.305420,-2637.366943,868.795349,724.495361,732.305237,1071.922241,-1733.774170,745.561707,675.807251,-1969.682495,1285.199829,855.414124,-1876.521240,895.916016,-1739.965576,887.414551,727.908020,933.638367,818.088013,-1785.304077,-2121.486572,-1180.591187,1017.250061,-2005.167236,-2621.274658,1962.813232,1934.798828,1862.521729,1260.327271,...,900.816040,-649.010742,580.633911,484.131775,845.387024,816.187439,-831.875366,-978.137207,656.135986,-710.609375,821.452271,-1120.232178,484.571198,595.803467,700.406494,-869.591064,-770.793823,-481.236389,573.154053,610.237122,657.373718,-561.213806,480.170776,-103.934235,-611.363647,-570.601501,-670.788635,-271.774841,434.906281,-390.098877,-299.975616,513.005066,551.919739,484.820129,196.327759,-291.994904,393.222443,225.378906,363.191650,255.296524
4,-705.918884,-645.627441,-441.810181,491.525452,-757.347778,287.098206,252.916092,282.970703,296.957520,-439.440674,521.248047,235.806305,

In [ ]:
b = song_meta_60000.sort_values('songnewid')
b.reset_index(inplace = True)
b

,index,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,songid,likecount,songnewid
0,932,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20101007,가을방학,1035872,[437760],가끔 미치도록 네가 안고 싶어질 때가 있어,"[GN0500, GN0800]",[가을방학],366786,352819,0
1,367,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20111010,Unplugged,2018706,[192827],"그대와 나, 설레임 (Feat. 소울맨)","[GN0500, GN0800]",[어쿠스틱 콜라보],133143,277721,1
2,1658,[GN0901],20140527,In The Lonely Hour,2258028,[718042],I`m Not The Only One,[GN0900],[Sam Smith],625875,277206,2
3,1613,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20100914,원모어찬스,1023955,[472980],널 생각해,"[GN0500, GN0800]",[원 모어 찬스 (one more chance)],610933,277176,3
4,1541,"[GN0101, GN0103]",19990100,A Night In Seoul,5422,[6017],여전히 아름다운지,[GN0100],[김연우],580074,272594,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1854,118,"[GN0105, GN0101]",20121005,기다림,2151973,[493783],혼자,[GN0100],[커피소년],43689,60142,1854
1855,1065,"[GN0501, GN0101, GN0506, GN0509]",20140307,안돼,2239069,[681456],안돼,"[GN0500, GN0100]",[스웨덴세탁소],409828,60123,1855
1856,1095,[GN0901],20180511,Voicenotes,10057083,[838654],Attention,[GN0900],[Charlie Puth],418694,60103,1856
1857,892,[GN1201],20090902,Party Rock,649652,[331467],Shots (Feat. Lil Jon),[GN1200],[LMFAO],349306,60052,1857


In [ ]:
c = b['songid']
a.columns = c.tolist()
a

,366786,133143,625875,610933,580074,116573,654757,140867,173943,207558,339802,117595,645489,88503,368753,1236,11657,459256,11924,505710,146989,395389,177460,253755,484234,464051,497066,663256,13198,635537,222763,581799,205757,705515,554106,675115,629738,629980,549178,144856,...,204888,88471,186320,143345,675366,126650,662661,369917,293267,72797,621690,397141,380410,626805,406498,113667,583792,192448,63146,90821,303123,39641,267593,238154,94648,647498,92908,483157,53295,426915,24121,547386,610218,470207,423647,43689,409828,418694,349306,156278
0,24.253807,-51.254528,-46.996181,22.569067,-71.503616,13.388608,12.332981,6.001461,13.366298,106.301331,31.064516,-98.443832,22.532354,57.792255,6.171138,9.599284,13.308151,-16.490648,11.675436,-27.039684,-31.215252,14.863453,11.425688,-17.813820,10.416659,-17.626663,11.600269,10.094570,11.633101,9.992886,72.556824,-62.836308,-70.312004,29.056932,-74.977005,60.691566,34.763988,20.759298,30.082634,16.844210,...,14.567137,88.178322,11.672685,-31.285986,32.076256,12.897902,57.011841,-59.644859,19.809523,-59.447575,12.047776,71.056282,-32.886795,10.073548,10.150181,-42.986034,-10.409890,-16.020340,3.109059,10.676986,6.262126,-45.910763,7.908444,12.076251,-21.100864,-35.184357,-27.313522,-53.212902,9.438646,42.398090,-30.230003,11.597405,-31.957111,6.898540,-46.485683,25.765547,12.713105,-39.970078,6.378341,-47.855728
1,-1470.929565,-1355.127563,-920.566895,1043.830811,-1586.635742,615.759155,542.872253,606.382629,633.666504,-907.982788,1109.625977,502.574860,-1640.494019,547.377014,455.903229,460.610596,671.367065,-1077.143066,465.658203,423.840729,-1224.640991,806.708252,537.316895,-1166.401123,562.356445,-1081.201660,556.476074,458.086487,585.741516,513.637329,-1105.975342,-1320.177734,-733.267273,638.273254,-1248.418701,-1627.323120,1229.539185,1211.728271,1166.943237,789.202148,...,565.284180,-395.697205,367.758148,305.977081,532.977600,513.626282,-511.950928,-605.787354,413.908905,-439.391907,517.157349,-690.811340,305.548248,375.504303,440.933838,-538.505127,-476.300964,-294.868561,361.701752,386.444885,415.353729,-345.264374,305.618561,-57.798676,-376.746918,-351.272430,-416.032013,-165.370071,279.384796,-234.541306,-182.555313,326.258575,348.598480,308.789825,126.894943,-174.333984,250.836243,145.915894,233.100632,162.703857
2,38.976582,-77.060745,-66.863953,39.218273,-104.646858,24.447001,23.030571,13.239692,24.569788,163.471664,51.005142,-139.776993,33.790161,89.366989,13.850610,18.779446,24.210264,-23.195904,20.307323,-35.579529,-44.954159,26.796583,21.443331,-26.152349,19.982233,-24.870432,21.692665,19.328588,21.963381,19.211355,111.385628,-92.920868,-101.624939,47.313442,-110.589973,91.345940,57.137215,36.306488,50.075703,30.292643,...,26.070877,134.185013,21.766052,-41.784912,51.717644,23.511215,89.554932,-84.633240,33.372822,-86.112328,22.775463,108.050591,-43.664967,19.303455,19.264071,-61.396000,-13.067863,-20.675570,8.816784,19.884951,13.307994,-65.717804,15.889306,22.052849,-28.253523,-52.007095,-40.865597,-76.187973,19.670727,68.342125,-42.743073,21.092951,-43.820976,14.198607,-65.744553,43.105110,22.443024,-54.794926,13.540292,-68.183197
3,-2371.488281,-2173.900146,-1482.632202,1668.704712,-2550.095947,980.166443,864.138245,966.156799,1011.174316,-1472.317993,1770.925781,804.305420,-2637.366943,868.795349,724.495361,732.305237,1071.922241,-1733.774170,745.561707,675.807251,-1969.682495,1285.199829,855.414124,-1876.521240,895.916016,-1739.965576,887.414551,727.908020,933.638367,818.088013,-1785.304077,-2121.486572,-1180.591187,1017.250061,-2005.167236,-2621.274658,1962.813232,1934.798828,1862.521729,1260.327271,...,900.816040,-649.010742,580.633911,484.131775,845.387024,816.187439,-831.875366,-978.137207,656.135986,-710.609375,821.452271,-1120.232178,484.571198,595.803467,700.406494,-869.591064,-770.793823,-481.236389,573.154053,610.237122,657.373718,-561.213806,480.170776,-103.934235,-611.363647,-570.601501,-670.788635,-271.774841,434.906281,-390.098877,-299.975616,513.0

In [ ]:
d = playlist_map_60000_newid_vallog['listid'].unique()
d

array([118598,  45144, 127575, ..., 122127,  77438,  65189])

In [ ]:
e = []
for i in tqdm(range(len(a))):
    f = ((a.iloc[i, :].sort_values(ascending = False)).index)[:100]
    f = f.tolist()
    e.append(f)

  0%|          | 0/12053 [00:00<?, ?it/s]

In [ ]:
g = pd.DataFrame({'id' : d, 'songs' : e})
g

,id,songs
0,118598,"[594116, 427246, 22573, 311, 518960, 207558, 2..."
1,45144,"[686809, 407044, 205247, 144632, 413952, 41318..."
2,127575,"[594116, 427246, 22573, 311, 207558, 518960, 2..."
3,53131,"[686809, 407044, 205247, 144632, 413952, 41318..."
4,44037,"[686809, 407044, 205247, 144632, 413952, 41318..."
...,...,...
12048,17766,"[686809, 407044, 205247, 144632, 413952, 41318..."
12049,101722,"[686809, 407044, 205247, 144632, 413952, 41318..."
12050,122127,"[686809, 407044, 205247, 144632, 413952, 41318..."
12051,77438,"[686809, 407044, 205247, 144632, 413952, 41318..."


-----------------------------------------------------

In [ ]:
result = pd.read_json("/content/drive/MyDrive/KUBIG 2021-2/추천시스템 프로젝트/리더보드 제출 모델 및 결과파일/word2vec_Khaii/results.json")
result

,id,songs,tags
0,118598,"[207912, 623047, 422438, 703323, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 디즈니OST, 영화음악, ..."
1,131447,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
2,51464,"[291080, 270647, 358818, 659416, 193610, 29532...","[발라드, 슬픔, 이별, 추억, 회상, 새벽, 밤, 잔잔한, 여행, 설렘]"
3,45144,"[144663, 367963, 357367, 351888, 576186, 19533...","[감성, 발라드, 사랑, 이별, 기분전환, 가을, 잔잔한, 새벽, 카페, 휴식]"
4,79929,"[412769, 211220, 70314, 242124, 445299, 106129...","[CCM, 찬양, 은혜, 예배, 교회, 국내ccm, 워십, 찬송, 제이어스, 감사]"
...,...,...,...
23010,101722,"[116573, 13142, 473514, 366786, 339802, 281936...","[새벽, 밤, 추억, 이별, 슬픔, 회상, 힐링, 휴식, 발라드, 기분전환]"
23011,122127,"[352228, 48209, 138932, 473514, 630552, 4173, ...","[추억, 회상, 사랑, 힐링, 휴식, 부모님, 엄마, 잔잔한, 새벽, 설렘]"
23012,77438,"[274504, 140867, 679436, 360062, 21125, 663256...","[팝, 팝송, 기분전환, Pop, 드라이브, 휴식, 힐링, 카페, 잔잔한, 감성]"
23013,36231,"[548041, 50031, 46497, 699175, 229703, 2826, 1...","[클래식, 잔잔한, 피아노, 아기클래식, 힐링, 조성진, 0살, 첼로, 기분전환, 휴식]"


In [ ]:
final = pd.merge(result, g, on = 'id', how = 'left')
final

,id,songs_x,tags,songs_y
0,118598,"[207912, 623047, 422438, 703323, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 디즈니OST, 영화음악, ...","[594116, 427246, 22573, 311, 518960, 207558, 2..."
1,131447,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]",NaN
2,51464,"[291080, 270647, 358818, 659416, 193610, 29532...","[발라드, 슬픔, 이별, 추억, 회상, 새벽, 밤, 잔잔한, 여행, 설렘]",NaN
3,45144,"[144663, 367963, 357367, 351888, 576186, 19533...","[감성, 발라드, 사랑, 이별, 기분전환, 가을, 잔잔한, 새벽, 카페, 휴식]","[686809, 407044, 205247, 144632, 413952, 41318..."
4,79929,"[412769, 211220, 70314, 242124, 445299, 106129...","[CCM, 찬양, 은혜, 예배, 교회, 국내ccm, 워십, 찬송, 제이어스, 감사]",NaN
...,...,...,...,...
23010,101722,"[116573, 13142, 473514, 366786, 339802, 281936...","[새벽, 밤, 추억, 이별, 슬픔, 회상, 힐링, 휴식, 발라드, 기분전환]","[686809, 407044, 205247, 144632, 413952, 41318..."
23011,122127,"[352228, 48209, 138932, 473514, 630552, 4173, ...","[추억, 회상, 사랑, 힐링, 휴식, 부모님, 엄마, 잔잔한, 새벽, 설렘]","[686809, 407044, 205247, 144632, 413952, 41318..."
23012,77438,"[274504, 140867, 679436, 360062, 21125, 663256...","[팝, 팝송, 기분전환, Pop, 드라이브, 휴식, 힐링, 카페, 잔잔한, 감성]","[686809, 407044, 205247, 144632, 413952, 41318..."
23013,36231,"[548041, 50031, 46497, 699175, 229703, 2826, 1...","[클래식, 잔잔한, 피아노, 아기클래식, 힐링, 조성진, 0살, 첼로, 기분전환, 휴식]",NaN


In [ ]:
final['songs'] = np.where(pd.notnull(final['songs_y']) == True, final['songs_y'], final['songs_x'])
final

,id,songs_x,tags,songs_y,songs
0,118598,"[207912, 623047, 422438, 703323, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 디즈니OST, 영화음악, ...","[594116, 427246, 22573, 311, 518960, 207558, 2...","[594116, 427246, 22573, 311, 518960, 207558, 2..."
1,131447,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]",NaN,"[144663, 116573, 357367, 366786, 654757, 13314..."
2,51464,"[291080, 270647, 358818, 659416, 193610, 29532...","[발라드, 슬픔, 이별, 추억, 회상, 새벽, 밤, 잔잔한, 여행, 설렘]",NaN,"[291080, 270647, 358818, 659416, 193610, 29532..."
3,45144,"[144663, 367963, 357367, 351888, 576186, 19533...","[감성, 발라드, 사랑, 이별, 기분전환, 가을, 잔잔한, 새벽, 카페, 휴식]","[686809, 407044, 205247, 144632, 413952, 41318...","[686809, 407044, 205247, 144632, 413952, 41318..."
4,79929,"[412769, 211220, 70314, 242124, 445299, 106129...","[CCM, 찬양, 은혜, 예배, 교회, 국내ccm, 워십, 찬송, 제이어스, 감사]",NaN,"[412769, 211220, 70314, 242124, 445299, 106129..."
...,...,...,...,...,...
23010,101722,"[116573, 13142, 473514, 366786, 339802, 281936...","[새벽, 밤, 추억, 이별, 슬픔, 회상, 힐링, 휴식, 발라드, 기분전환]","[686809, 407044, 205247, 144632, 413952, 41318...","[686809, 407044, 205247, 144632, 413952, 41318..."
23011,122127,"[352228, 48209, 138932, 473514, 630552, 4173, ...","[추억, 회상, 사랑, 힐링, 휴식, 부모님, 엄마, 잔잔한, 새벽, 설렘]","[686809, 407044, 205247, 144632, 413952, 41318...","[686809, 407044, 205247, 144632, 413952, 41318..."
23012,77438,"[274504, 140867, 679436, 360062, 21125, 663256...","[팝, 팝송, 기분전환, Pop, 드라이브, 휴식, 힐링, 카페, 잔잔한, 감성]","[686809, 407044, 205247, 144632, 413952, 41318...","[686809, 407044, 205247, 144632, 413952, 41318..."
23013,36231,"[548041, 50031, 46497, 699175, 229703, 2826, 1...","[클래식, 잔잔한, 피아노, 아기클래식, 힐링, 조성진, 0살, 첼로, 기분전환, 휴식]",NaN,"[548041, 50031, 46497, 699175, 229703, 2826, 1..."


In [ ]:
real_final = pd.DataFrame(final[['id','songs','tags']])
real_final

,id,songs,tags
0,118598,"[594116, 427246, 22573, 311, 518960, 207558, 2...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 디즈니OST, 영화음악, ..."
1,131447,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
2,51464,"[291080, 270647, 358818, 659416, 193610, 29532...","[발라드, 슬픔, 이별, 추억, 회상, 새벽, 밤, 잔잔한, 여행, 설렘]"
3,45144,"[686809, 407044, 205247, 144632, 413952, 41318...","[감성, 발라드, 사랑, 이별, 기분전환, 가을, 잔잔한, 새벽, 카페, 휴식]"
4,79929,"[412769, 211220, 70314, 242124, 445299, 106129...","[CCM, 찬양, 은혜, 예배, 교회, 국내ccm, 워십, 찬송, 제이어스, 감사]"
...,...,...,...
23010,101722,"[686809, 407044, 205247, 144632, 413952, 41318...","[새벽, 밤, 추억, 이별, 슬픔, 회상, 힐링, 휴식, 발라드, 기분전환]"
23011,122127,"[686809, 407044, 205247, 144632, 413952, 41318...","[추억, 회상, 사랑, 힐링, 휴식, 부모님, 엄마, 잔잔한, 새벽, 설렘]"
23012,77438,"[686809, 407044, 205247, 144632, 413952, 41318...","[팝, 팝송, 기분전환, Pop, 드라이브, 휴식, 힐링, 카페, 잔잔한, 감성]"
23013,36231,"[548041, 50031, 46497, 699175, 229703, 2826, 1...","[클래식, 잔잔한, 피아노, 아기클래식, 힐링, 조성진, 0살, 첼로, 기분전환, 휴식]"


In [ ]:
res_lst = []

for i in range(real_final.shape[0]):
  res_lst.append({
                "id": real_final['id'][i],
                "songs": real_final['songs'][i],
                "tags": real_final['tags'][i]
            })

In [ ]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np


def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./arena_data/" + parent)
    with io.open("./arena_data/" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [ ]:
write_json(res_lst, 'results/results.json')